## Brief

**Project goal**: to analyze the data of one of the services competing in book market. Database contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product. 

**Tasks:**

- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating (look only at books with at least 50 ratings).
- Find the average number of text reviews among users who rated more than 50 books.



In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Study the table "authors"

query = "SELECT * FROM authors LIMIT 5"

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Study the table "books"

query = "SELECT * FROM books LIMIT 5"

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
# Study the table "publishers"

query = "SELECT * FROM publishers LIMIT 5"

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Study the table "ratings"

query = "SELECT * FROM ratings LIMIT 5"

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Study the table "reviews"

query = "SELECT * FROM reviews LIMIT 5"

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


We have 5 datasets with data on books, their authors, on publishers, reviews and ratings.

- Books contain data on book id, author id, title, number of pages, publication date and publisher id;
- Authors contain data on author id and author name;
- Publishers: publisher id and publisher name;
- Ratings: rating id, book id, username, rating;
- Reviews: review id, book id, username and text of the review.

In [8]:
# Find the number of books released after January 1, 2000.

query = "SELECT COUNT(Distinct book_id) FROM books WHERE publication_date > '2000-01-01'"

pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 books were released after January 1, 2000.

In [9]:
# Find the number of user reviews and the average rating for each book.

query = f"""
SELECT books.title, AVG(rating) AS avg_rating, COUNT(DISTINCT review_id) AS n_reviews 
FROM ratings 
LEFT JOIN reviews ON reviews.book_id = ratings.book_id 
LEFT JOIN books On books.book_id = reviews.book_id
GROUP BY ratings.book_id, books.title 
ORDER BY  avg_rating DESC"""

pd.io.sql.read_sql(query, con = engine)

,title,avg_rating,n_reviews
0,Arrows of the Queen (Heralds of Valdemar #1),5.00,2
1,The Walking Dead Book One (The Walking Dead #...,5.00,2
2,Light in August,5.00,2
3,Wherever You Go There You Are: Mindfulness Me...,5.00,2
4,Captivating: Unveiling the Mystery of a Woman'...,5.00,2
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,2.25,3
996,His Excellency: George Washington,2.00,2
997,Drowning Ruth,2.00,3
998,Junky,2.00,2


In [11]:
query = f"""
SELECT publisher, COUNT(DISTINCT book_id) AS n_books
FROM publishers AS p 
JOIN books AS b ON b.publisher_id = p.publisher_id 
WHERE num_pages > 50 
GROUP BY publisher 
ORDER BY COUNT(DISTINCT book_id) DESC LIMIT 1"""

pd.io.sql.read_sql(query, con = engine)

,publisher,n_books
0,Penguin Books,42


The publisher that has released the greatest number of books with more than 50 pages is Penguin Books.

In [13]:
#Identify the author with the highest average book rating (look only at books with at least 50 ratings).

query = f"""
WITH new_ratings AS (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(*) >= 50),
                    
book_rating AS (SELECT book_id,
                       AVG(rating) AS avg_rating
                FROM ratings
                WHERE book_id IN (SELECT book_id FROM new_ratings)
                GROUP BY book_id)
                
SELECT author,
       AVG(avg_rating) avg_author_rating,
       COUNT(DISTINCT book_id) n_books
FROM authors
INNER JOIN books USING(author_id)
INNER JOIN book_rating USING(book_id)
GROUP BY author
ORDER BY avg_author_rating DESC
LIMIT 1;
"""

pd.io.sql.read_sql(query, con = engine)

,author,avg_author_rating,n_books
0,J.K. Rowling/Mary GrandPré,4.283844,4


The author with the highest average book rating is J.K. Rowling/Mary GrandPré.

In [14]:

# Find the average number of text reviews among users who rated more than 50 books.

query = f"""
WITH users_more_than_50_books AS (SELECT username
                                  FROM ratings
                                  GROUP BY username
                                  HAVING COUNT(DISTINCT book_id) > 50),
reviews_per_user AS (SELECT username,
                            COUNT(*) n_reviews
                     FROM reviews
                     WHERE username IN (SELECT username FROM users_more_than_50_books)
                     GROUP BY username)
                     
SELECT ROUND(AVG(n_reviews),2)
FROM reviews_per_user

""" 

pd.io.sql.read_sql(query, con = engine)

,round
0,24.33


The average number of text reviews among users who rated more than 50 books is 24.33.

**Conclusions**

- We have 5 datasets with data on books, their authors, on publishers, reviews and ratings.
- 819 books were released after January 1, 2000;
- The publisher that has released the greatest number of books with more than 50 pages is Penguin Books;
- The author with the highest average book rating is J.K. Rowling/Mary GrandPré;
- The average number of text reviews among users who rated more than 50 books is 24.33.